# Linear Sysetms Tutorial

Original authors: David Heeger, Eero Simoncelli, and Patrick Teo 6/96

Based on OBVIUS tutorial by David Heeger and Eero Simoncelli

Translated to Python by Michael Waskom 8/19.

In [ ]:
import numpy as np
from numpy.fft import fft, fftshift, ifftshift
from scipy import signal, ndimage, linalg
import matplotlib.pyplot as plt
%matplotlib inline

## Discrete-time sequences

Discrete-time sequences are represented as a sequence of numbers $f[n]$, for integer values of $n$. There are several important, basic sequences. Examples of these basic sequences will be plotted below. We now plot several of the important sequences, from $n=1$ to $n=32$.


In [ ]:
n_range = np.arange(32)

### Impulse sequence, delta[n]

Is 1 only when n = 0. Here we plot delta[n - 16]:

In [ ]:
impulse = n_range == 16
f, ax = plt.subplots()
ax.plot(n_range, impulse, marker="o")
f.tight_layout()

### Step sequence, u[n]

Is 0 when n < 0. Here we plot u[n - 16]:

In [ ]:
step = n_range >= 16
f, ax = plt.subplots()
ax.plot(n_range, step, marker="o")
f.tight_layout()

### Sinusoidal sequence with period 8

In [ ]:
amplitude, phase_off, period = 1, 0, 8
freq = 2 * np.pi / period
sinusoid = amplitude * np.sin(freq * n_range + phase_off)

f, ax = plt.subplots()
ax.plot(n_range, sinusoid, marker="o")

# Notice that adding $2\pi$ to the frequency gives the same sinusoid
freq = 2 * np.pi / period + 2 * np.pi
sinusoid_shift_2pi = amplitude * np.sin(freq * n_range + phase_off)
ax.plot(n_range, sinusoid, marker=".", linestyle="--")

f.tight_layout()

The importance of this is that we need only consider frequencies in a frequency interval of length $2\pi$ such as $-\pi$ to $\pi$. Also notice that although continuous sinusoids with frequency $\omega$ are periodic with period $2\pi/\omega$, this is not necessarily true of discrete sinusoids. For example, a discrete sinusoid with frequency $\omega=1$ is NOT periodic:

In [ ]:
period = 2 * np.pi
freq = 2 * np.pi / period

non_periodic_sinusoid = amplitude * np.sin(freq * n_range + phase_off)

f, ax = plt.subplots()
ax.plot(n_range, non_periodic_sinusoid, marker="o")
f.tight_layout()

Why isn't this sequence periodic?  Is it because we've plotted only 32 samples?  If we were to plot more samples, would it ever repeat?

---

For a finite length sequence, we have an even more stringent requirement.  By a periodic finite length sequence, we mean circularly periodic.  When you go off the end you start back at the beginning.  To be periodic, the sequence length must be a multiple of the period.

Altogether, there are only N distinguishable frequencies that are circularly periodic with period (sequence length) N.  These frequencies are:
$$2\pi k/N \quad\mathrm{for}\quad k=0\,,1\ldots,\,N-1.$$

In our examples, N=32, so the frequencies are:
$$0,\, \pi/16,\, 2\pi/16,\, 3\pi/16,\ldots,\,31\pi/16.$$

This set of discrete (co-)sinusoids can also be indexed in another way:
$$2\pi k/N \quad\mathrm{for}\quad k=-N/2,\ldots,\,-1,\,0,\,1\ldots,\,N/2.$$

In our examples, these periods are:
$$-16\pi/16,\ldots,\,\pi/16,\,0,\,\pi/16\ldots,\,15\pi/16.$$

Take a look at some of these sinusoids and cosinusoids to see that these frequencies are all distinct.

In [ ]:
## Enter code here

Are the sinusoids and cosinusoids with frequencies 0 and $\pi$ distinct?  How about with frequencies $-\pi$ and $\pi$?

----

## Linear systems

A discrete time system is defined as a transformation or operator that maps an input sequence to an output sequence:

$$f[x] \rightarrow g[x] \quad\textrm{or}\quad g[x] = T\{f[x]\}$$

Linear systems are defined by the principle of superposition. Superposition has two parts: additivity and homogeneity.

**Additivity:** $T\{f_1[x] + f_2[x]\} = T\{f_1[x]\} + T\{f_2[x]\}$.

**Homogeneity (scaling):** $T\{a\,f[x]\} = a\,T\{f[x]\}$.

A linear system can be expressed as a matrix multiplication:
$$g[x] = M\,f[x],$$
where $M$ is an $m \times n$ matrix, $g[x]$ is a sequence of length $m$, and $f[x]$ is a sequence of length $n$.

A time-invariant system is one for which a shift or delay of the input sequence causes a corresponding shift in the output sequence. An example of a linear time-*variant* system is subsampling. We'll get to that later.

For a linear time-invariant system, the rows of the $M$ matrix are all shifted copies of each other. Such a matrix is called a "Toeplitz" matrix. The output of a linear time-invariant sysetm can also be computed using convolution. Convolution is equivalent to matrix-multiplication when using a Toeplitz matrix.

First, let's create a Toeplitz matrix and display it as an image;

In [ ]:
Tmatrix = np.zeros((32, 32))
filt = [-.004, -.031, -.047, .031, .102, .031, -.047, -.031, -.004]
for row in range(32):
    for k, col in enumerate(range(row - 4, row + 4 + 1)):
        Tmatrix[row, (col + 32) %  32] = filt[k]
        
f, ax = plt.subplots(figsize=(4, 4))
ax.matshow(Tmatrix, cmap="gray")
f.tight_layout()

Now, let's take our impulse signal and multiply it through the matrix:

In [ ]:
impulse_response = Tmatrix @ impulse
f, ax = plt.subplots()
ax.plot(n_range, impulse_response, "o-")
f.tight_layout()

Matrix multiplication is an inefficient way of doing the above computation because most of the entries in the matrix are zeros. Like `MATLAB`, `numpy` and `scipy` provide facilities for efficient linear filterin. A linear filter has only the interesting (non-zero) entries of the Toeplitz matrix. Then the output is computed using convolution, shifting the filter over the input signal. The efficiency of convolution (over matrix multiplicatoin) will be critical when we get to 2D linear transforms on images, and 3D (space-time) transform on image sequences. The matrices would be huge and very sparse (lots of zeros). Let's do the above transform again, using convolution:

In [ ]:
new_impulse_response = np.convolve(impulse, filt, "same")
ax.plot(n_range, new_impulse_response, ".--")
display(f)

### Sum of two impulses

A linear time-invariant system is completely characterized by its impulse response; that is, its response to an impulse input. The response to an impulse is the corresponding column in the Toeplitz matrix. Given the impulse response, we can compute the response to any input. Any input can be expressed as the sum of a bunch of scaled impulses. Since the system is linear, the output is the sum of a bunch of scaled copies of the impulse response. Example:

In [ ]:
impulse_10 = n_range == 10
impulse_15 = n_range == 15
signal = 5 * impulse_10 + 13 * impulse_15

f, ax = plt.subplots()
ax.plot(n_range, signal, "o-")
f.tight_layout()

Filter responses to individual impulses:

In [ ]:
impulse_response_10 = np.convolve(impulse_10, filt, "same")
impulse_response_15 = np.convolve(impulse_15, filt, "same")
f, ax = plt.subplots()
ax.plot(n_range, impulse_response_10, "o-")
ax.plot(n_range, impulse_response_15, "o-")
f.tight_layout()

Filter response to scaled sum of impulses:

In [ ]:
signal_response = np.convolve(signal, filt, "same")
ax.plot(n_range, signal_response, "o-")
display(f)

Sum of scaled filter responses to individual inputs:

In [ ]:
sum_of_impulse_responses = 5 * impulse_response_10 + 13 * impulse_response_15
assert np.allclose(signal_response, sum_of_impulse_responses)
ax.plot(n_range, sum_of_impulse_responses, "kx")
display(f)

As another example, consider the filter response to the step"

In [ ]:
step_response = np.convolve(step, filt, "same")
f, ax = plt.subplots()
ax.plot(n_range, step_response, "o-")
f.tight_layout()

Represent filter response to step as sum of filter responses to impulses:

In [ ]:
sum_of_impulse_responses = np.zeros(step.shape)
for n in range(16, 33):
    impulse_n = n_range == n
    sum_of_impulse_responses += np.convolve(impulse_n, filt, "same")

assert np.allclose(step_response, sum_of_impulse_responses)
    
f, ax = plt.subplots()
ax.plot(n_range, sum_of_impulse_responses, "o-")
f.tight_layout()

Here's a tooling detail: the matlab version of the tutorial has been using circular convolution and has a note here about how the step response makes the "wrapping" inherent to circular convolution apparent. We've been using the most basic convolution function in numerical Python, `numpy.convolve`, which does not implement circular convolution (as of this writing). For more options we can turn to `scipy.ndimage.convolve` function. You'll see that we imported the `ndimage` library from `scipy` at the top of the notebook. Two differences are worth noting.

- First, our `step` array actually has a boolean data type (look above to how we defined it). The `numpy` function casts it properly to a `float` data type before convolution, but we need to handle that ourselves with the `scipy` function.
- Second, the `scipy` function is not just a drop-in replacement: its signature is a little bit different, and `mode` is not the third parameter, so we need to be more explicit.

In [ ]:
step_float = step.astype(np.float)
step_response_circ = ndimage.convolve(step_float, filt, mode="wrap")
f, ax = plt.subplots()
ax.plot(n_range, step_response_circ, "o-")
f.tight_layout()

Now we can see the effects of circular convolution, which "wraps" around from the end of the sequence back to the beginning, as if the sequence was a full period of a longer periodic sequence. The consequence of circular convolution is evidence in the step-response. This is one way to handle the endpoints of the sequence. Inspecting the help for `ndimage.convolve` provides a few other options, which I am duplicating here:

```
The `mode` parameter determines how the input array is extended
when the filter overlaps a border. By passing a sequence of modes
with length equal to the number of dimensions of the input array,
different modes can be specified along each axis. Default value is
'reflect'. The valid values and their behavior is as follows:

'reflect' (`d c b a | a b c d | d c b a`)
    The input is extended by reflecting about the edge of the last
    pixel.

'constant' (`k k k k | a b c d | k k k k`)
    The input is extended by filling all values beyond the edge with
    the same constant value, defined by the `cval` parameter.

'nearest' (`a a a a | a b c d | d d d d`)
    The input is extended by replicating the last pixel.

'mirror' (`d c b | a b c d | c b a`)
    The input is extended by reflecting about the center of the last
    pixel.

'wrap' (`a b c d | a b c d | a b c d`)
    The input is extended by wrapping around to the opposite edge.
```

## Properties of linear time-invariant systems

Convolution is commutative (i.e. the order of the two consecutive convolution operations is irrelevant):

In [ ]:
filt1 = filt.copy()
filt2 = np.array([1, 4, 6, 4, 1]) / 16

resp_1_2 = ndimage.convolve(ndimage.convolve(step_float, filt1, mode="wrap"), filt2, mode="wrap")
resp_2_1 = ndimage.convolve(ndimage.convolve(step_float, filt2, mode="wrap"), filt1, mode="wrap")

f, ax = plt.subplots()
ax.plot(n_range, resp_1_2, "-")
ax.plot(n_range, resp_2_1, "o")
f.tight_layout()

*n.b.:* even though convolution (shift-invariant linear systems) commute, not all linear systetms commute. For example, matrix multiplication is not, in general, commutative.

Convolution also follows the distributive property (i.e. the sum of convolutions with two filters equals the convolution with the sum of the filters).

In [ ]:
filt2 = np.array([0, 0, 1, 4, 6, 4, 1, 0, 0]) / 16
sum_of_filters = filt1 + filt2

resp1 = ndimage.convolve(step_float, filt1, mode="wrap")
resp2 = ndimage.convolve(step_float, filt2, mode="wrap")
sum_of_responses = resp1 + resp2

response_to_sum_of_filters = ndimage.convolve(step_float, sum_of_filters, mode="wrap")

assert np.allclose(response_to_sum_of_filters, sum_of_responses)

f, ax = plt.subplots()
ax.plot(n_range, sum_of_responses, "-")
ax.plot(n_range, response_to_sum_of_filters, "o")
f.tight_layout()

## Invertible linear systems

Here's a simple example of a linear filter that shifts the sequence by two samples. This operation can, of course, be inverted by shifting in the other direction.

In [ ]:
shift_filt = np.array([0, 0, 0, 0, 1])
unshift_filt = np.array([1, 0, 0, 0, 0])

shifted_sinusoid = ndimage.convolve(sinusoid, shift_filt, mode="wrap")
unshifted_sinusoid = ndimage.convolve(shifted_sinusoid, unshift_filt, mode="wrap")

f, ax = plt.subplots()
ax.plot(n_range, shifted_sinusoid, "o-")
ax.plot(n_range, unshifted_sinusoid, "o-")
f.tight_layout()

Another way to think of inverting a linear transform is in terms of inverting the corresponding transform matrix. The matrix for the shifting operation looks like this:

In [ ]:
Smatrix = np.zeros((32, 32))
one_row = [1, 0, 0, 0, 0]
for row in range(32):
    for k, col in enumerate(range(row - 2, row + 3)):
        Smatrix[row, col % 32] = one_row[k]

f, ax = plt.subplots(figsize=(4, 4))
ax.matshow(Smatrix, cmap="gray")
f.tight_layout()

Let's recompute the shifted sequences, using matrix multiplication instead of convolution:

In [ ]:
new_shifted_sinusoid = Smatrix @ sinusoid
f, ax = plt.subplots()
ax.plot(n_range, shifted_sinusoid, "-")
ax.plot(n_range, new_shifted_sinusoid, "o")
f.tight_layout()

To invert the transform, just invert the matrix and do another matrix multiplication:

In [ ]:
Smatrix_inv = np.linalg.inv(Smatrix)
f, ax = plt.subplots(figsize=(4, 4))
ax.matshow(Smatrix_inv, cmap="gray")
f.tight_layout()

In [ ]:
new_unshifted_sinusoid = Smatrix_inv @ shifted_sinusoid
f, ax = plt.subplots()
ax.plot(n_range, unshifted_sinusoid, "-")
ax.plot(n_range, new_unshifted_sinusoid, "o")
f.tight_layout()

### Another example

Here's another example of inverting a linear transform. In this example, we split a signal into two bands, a high frequency (or "highpass") and a low frequency (or "lowpass") band. Each band has the same number of samples as the original sequence, so the entire transform has twice as many samples as the original. The lowpass and highpass filters are carefully chosen so that summing the low and high bands reconstructs the original signal.

In [ ]:
lo_filt = [.008, .031, .094, .219, .297, .219, .094, .031, .008]
hi_filt = [-.008, -.031, -.094, -.219, .703, -.219, -.094, -.031, -.008]
x = np.linspace(-4, 4, len(lo_filt))

f, axes = plt.subplots(2, sharex=True, sharey=True)
axes[0].plot(x, lo_filt)
axes[1].plot(x, hi_filt)
f.tight_layout()

In [ ]:
lo_impulse_resp = np.convolve(impulse, lo_filt, "same")
hi_impulse_resp = np.convolve(impulse, hi_filt, "same")

f, axes = plt.subplots(2, sharex=True, sharey=True)
axes[0].plot(n_range, lo_impulse_resp, "o-")
axes[1].plot(n_range, hi_impulse_resp, "o-")
f.tight_layout()

In [ ]:
reconstructed_impulse = lo_impulse_resp + hi_impulse_resp
assert np.allclose(impulse, reconstructed_impulse)

f, ax = plt.subplots()
ax.plot(n_range, reconstructed_impulse, "o-")
f.tight_layout()

### Linear system responses to a sinusoid


In [ ]:
lo_sin_response = ndimage.convolve(sinusoid, lo_filt, mode="wrap")
hi_sin_response = ndimage.convolve(sinusoid, hi_filt, mode="wrap")

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(n_range, lo_sin_response, marker="o")
axes[1].plot(n_range, hi_sin_response, marker="o", color="C1")
f.tight_layout()

In [ ]:
reconstructed_sin = lo_sin_response + hi_sin_response

f, ax = plt.subplots()
ax.plot(n_range, reconstructed_sin, marker="o", mec="C1", mfc="C1")

assert np.allclose(reconstructed_sin, sinusoid)

Now let's see what the matrix looks like for this transform:

In [ ]:
n = 32
lh_matrix = np.zeros((2 * n, n))
for i in range(n):
    for k, j in enumerate(range(i - 4, i + 5)):
        lh_matrix[i, (j + n) %  n] = lo_filt[k]
        lh_matrix[i + n, (j + n) %  n] = hi_filt[k]

f, ax = plt.subplots(figsize=(4, 8))
ax.matshow(lh_matrix, cmap="gray")
f.tight_layout()

The top half of the rectangular matrix represents the shifted filter coefficients for the low-pass filter. The bottom half represents those of the hi-pass filter.

To invert the transform, we can use the pseudo-inverse, $(M^T M)^{-1} M^T$, where M is the matrix, $M^T$ is the matrix transpose, and the $^{-1}$ indicates matrix inverse.



In [ ]:
lh_matrix_inv = np.linalg.inv(lh_matrix.T @ lh_matrix) @ lh_matrix.T

In [ ]:
f, ax = plt.subplots(figsize=(8, 4))
ax.matshow(lh_matrix_inv, cmap="gray")
f.tight_layout()

Let's check that this really is the inverse:

In [ ]:
assert np.allclose(lh_matrix_inv @ lh_matrix, np.eye(n))

Now, recompute the transform (the lo-pass and hi-pass transform coefficients are displayed in one double-length sequence, next to each other):

In [ ]:
impulse_transform = lh_matrix @ impulse.T

f, ax = plt.subplots()
ax.plot(impulse_transform)
f.tight_layout()

Now, invert the transform:

In [ ]:
new_reconstruct_impulse = lh_matrix_inv @ impulse_transform

f, ax = plt.subplots()
ax.plot(new_reconstruct_impulse, "o-")
f.tight_layout()

assert np.allclose(impulse, new_reconstruct_impulse)

The inverse that we just used is different from just adding together the coefficients of the two bands (used above). There is more than one way to invert an overcomplete transform. Let's construct the matrix that corresponds to adding the coefficients from the two bands, and make sure that it also inverts the transform:

In [ ]:
new_lh_matrix_inv = np.c_[np.eye(n), np.eye(n)]

f, ax = plt.subplots(figsize=(8, 4))
ax.matshow(new_lh_matrix_inv, cmap="gray")
f.tight_layout()

Check that this is also an inverse:

In [ ]:
assert np.allclose(new_lh_matrix_inv @ lh_matrix, np.eye(n))

## Sinusoidal sequences

Sinusoidal and cosinusoidal sequences play a particularly important role in representing signals, because complex exponential sequences (including sines and cosines) are the eigenfunctions of finite-dimensional linear time-invariant systems.  For example, a sinusoidal sequence convolved with a linear filter gives another sinusoidal sequence of the same frequency.  Only the phase and amplitude of the output sinusoid will be different.


In [ ]:
f, ax = plt.subplots()
ax.plot(sinusoid, "o-")
result = ndimage.convolve(sinusoid, filt, mode="wrap")
ax.plot(result, "o-")
f.tight_layout()

That filter changes only the amplitude, not the phase. Here's one that also changes the phase (via shift/delay):

In [ ]:
shift_filt = np.r_[0, 0, filt]
result = ndimage.convolve(sinusoid, shift_filt, mode="wrap")
ax.plot(result, "o-")
display(f)

Let's try some really weird (in fact random) filters, just to demonstrate that no matter what you use, you still get a sinusoid of the same frequency:

In [ ]:
results = np.zeros((10, n))
for i in range(len(results)):
    filt = np.random.rand(5)
    results[i] = ndimage.convolve(sinusoid, filt, mode="wrap")

f, ax = plt.subplots()
ax.plot(results.T, "o-")
f.tight_layout()

## Fourier series representation

Any signal can be expressed as a (weighted) linear sum of impulses. Likewise, a signal can be expressed as a (weighted) linear sum of sines and cosines.

### Gaussian example

Example: make a gaussaian as a sum of cosines.

In [ ]:
gaussian = np.exp(-(n_range - n / 2) ** 2 / (2 * 4 ** 2))
f, ax = plt.subplots()
ax.plot(gaussian, "o-")
f.tight_layout()

Fourier series approximation of a gaussian:

In [ ]:
coefs = [.33, .470, .166, .029, .0025]
gaussian_series = np.sum([
    coef * np.cos(2 * np.pi * i / n * (n_range - n / 2))
    for i, coef in enumerate(coefs)
], axis=0)

f, ax = plt.subplots()
ax.plot(gaussian_series, "o-")
f.tight_layout()

assert np.allclose(gaussian, gaussian_series, atol=.05)

## Fourier transform

The Fourier transform is a particular linear transform that is used to compute the Fourier coefficients of a signal. The transform matrix looks like this:

In [ ]:
fourier_mat = np.zeros((n * 2, n))
for k in n_range:
    fourier_mat[k] = np.cos(2 * np.pi * k / n * n_range) / np.sqrt(n)
    fourier_mat[k + n] = np.sin(-2 * np.pi * k / n * n_range) / np.sqrt(n)

f, ax = plt.subplots(figsize=(4, 8))
ax.matshow(fourier_mat, cmap="gray")
f.tight_layout()

Each row is a sine or cosine. Rows 2 and 6, for example:

In [ ]:
f, ax = plt.subplots()
ax.plot(fourier_mat[2], "o-")
ax.plot(fourier_mat[6], "o-")
f.tight_layout()

The top half of the matrix contains cosines of various frequencies, and the bottom half contains sines of various frequencies. The very top row is a constant that pulls out the average (dc) component of a signal. The transform is self-inverting, that is, multiplying the matrix by its transpose gives the identity transform.

In [ ]:
fourier_inv_mat = fourier_mat.T
f, ax = plt.subplots(figsize=(8, 4))
ax.matshow(fourier_inv_mat, cmap="gray")
f.tight_layout()

In [ ]:
assert np.allclose(fourier_inv_mat @ fourier_mat, np.eye(n))

Let's look at the rows of the inverse (transpose) matrix. The first half of each row is a cosine, and the second half of each row is a sine. These sines and cosines are the same as the rows of the the Fourier matrix.

In [ ]:
f, ax = plt.subplots()
ax.plot(fourier_inv_mat[2, :n], "o-")
ax.plot(fourier_inv_mat[2, n:], "o-")
f.tight_layout()

Let's take the Fourier transform of a cosinusoid.

In [ ]:
num_cycles = 4
cosinusoid = np.cos(2 * np.pi * num_cycles / n * n_range)
ft_cosinusoid = fourier_mat @ cosinusoid

f, axes = plt.subplots(2)
axes[0].plot(cosinusoid, "o-")
axes[1].plot(ft_cosinusoid, "o-")
f.tight_layout()

We get a pair of impulses in the transform.  One of the impulses corresponds to the frequency of the signal (4 cycles per image) at position 4 in the transform.  **Why is there a second impulse?**

The Fourier transform is really set up to analyze complex signals.  For real signals, the transform has certain symmetry properties.  We will go into those in more detail below.

First, try a different frequency:

In [ ]:
num_cycles = 8
cosinusoid = np.cos(2 * np.pi * num_cycles / n * n_range)
ft_cosinusoid = fourier_mat @ cosinusoid

f, axes = plt.subplots(2)
axes[0].plot(cosinusoid, "o-")
axes[1].plot(ft_cosinusoid, "o-")
f.tight_layout()

For a sinusoid, we get impulses in the second half (positions > 32) of the output because the sinusoids are in the bottom half of the system matrix.

In [ ]:
transform = fourier_mat @ sinusoid

f, axes = plt.subplots(2)
axes[0].plot(sinusoid, "o-")
axes[1].plot(transform, "o-")
f.tight_layout()

The Fourier transform is inverted by using the transpose of the system matrix:

In [ ]:
invert_transform = fourier_mat.T @ transform
assert np.allclose(invert_transform, sinusoid)

axes[0].plot(invert_transform, ".--")
display(f)

## FFTs

The FFT (Fast Fourier Transform) is an efficient way of computing the Fourier transform, without bothering with the matrix multiplication.

In [ ]:
ft_sinusoid = fft(sinusoid)

f, axes = plt.subplots(2, sharey=True)
axes[0].plot(ft_sinusoid.real, "o-")
axes[1].plot(ft_sinusoid.imag, "o-")
f.tight_layout()

FFT returns complex numbers. The imaginary part contains the sine components and the real-part contains the cosine components. The first sample corresponds to the "DC" or constant coefficient.

Often, people represent Fourier coefficients in terms of magnitude and phase, rather than real and imaginary:

In [ ]:
mag_sin = np.abs(ft_sinusoid)

f, ax = plt.subplots()
ax.plot(mag_sin, "o-")
f.tight_layout()

Here's a whole series of Fourier transforms for different frequency sinusoids by looping through all of the possible frequencies of periodic sinuoisides.

In [ ]:
f, ax = plt.subplots()
colors = plt.cm.coolwarm(np.linspace(.1, .9, n))
for freq in n_range:
    sin_freq = np.sin(2 * np.pi * freq / 32 * n_range)
    imag_ft_sin_freq = fft(sin_freq).imag
    ax.plot(imag_ft_sin_freq, "o-", color=colors[freq])
f.tight_layout()

## Fourier transforms of other signals

Now, let's compute Fourier transforms of some other signals.

First, Fourier transform of a constant function:

In [ ]:
constant = np.ones(n)
mag_constant = np.abs(fft(constant))

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(constant, "o-")
axes[1].plot(mag_constant, "o-")
f.tight_layout()

The `fftshift` function shifts the frequency axis to put the dc component in the middle:

In [ ]:
mag_constant = fftshift(mag_constant)
f, axes = plt.subplots(2, sharex=False)
axes[0].plot(constant, "o-")
axes[1].plot(n_range - n / 2, mag_constant, "o-")
f.tight_layout()

## Fourier transform of a Gaussian function

In the Fourier transform of a Gaussian, the real part is itself a Gaussian and the imaginary part is zero:

In [ ]:
gaussian = np.exp(-((n_range - 16) / 6) ** 2)
ft_gaussian = fftshift(fft(ifftshift(gaussian)))

f, axes = plt.subplots(2, sharex=True)

axes[0].plot(gaussian, "o-")
axes[1].plot(ft_gaussian.real, "o-")
axes[1].plot(ft_gaussian.imag, "o-")
f.tight_layout()

Making the Gaussian smaller in one domain makes it larger in the other domain:

In [ ]:
little_gauss = np.exp(-((n_range - 16) ** 2 / 2))
ft_little_gauss = fftshift(fft(ifftshift(little_gauss)))

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(little_gauss, "o-")
axes[1].plot(ft_little_gauss.real, "o-")
f.tight_layout()

## Symmetry properties of the Fourier transform

For any real-valued, antisymmetric (odd) function, in with $f(x) = -f(-x)$, the real part of the FT is zero, and the imaginary part of the FT is antisymmetric (odd). For any real-valued, symmetric (even) function, in which $f(x) = f(-x)$, the imaginary part of the FT is zero and the real part is symmetric (even).

To show this, we decompose a random signal into its even and odd components:

In [ ]:
even_odd_x = np.arange(-n, n + 1)
random_signal = .5 - np.random.rand(even_odd_x.size)

even_signal = .5 * (random_signal + random_signal[::-1])
assert np.allclose(even_signal, even_signal[::-1])

odd_signal = .5 * (random_signal - random_signal[::-1])
assert np.allclose(odd_signal, -odd_signal[::-1])

assert np.allclose(random_signal, even_signal + odd_signal)

In [ ]:
f, axes = plt.subplots(3, sharex=True, sharey=True)
axes[0].plot(even_odd_x, random_signal, "o-")
axes[1].plot(even_odd_x, even_signal, "o-")
axes[2].plot(even_odd_x, odd_signal, "o-")
f.tight_layout()

In [ ]:
ft_even_signal = fftshift(fft(ifftshift(even_signal)))
ft_odd_signal = fftshift(fft(ifftshift(odd_signal)))

f, axes = plt.subplots(2, sharex=True, sharey=True)
axes[0].plot(even_odd_x, ft_even_signal.real, "o-")
axes[0].plot(even_odd_x, ft_even_signal.imag, "o-")
axes[1].plot(even_odd_x, ft_odd_signal.real, "o-")
axes[1].plot(even_odd_x, ft_odd_signal.imag, "o-")
f.tight_layout()

For any real-valued signal, the real part of the FT is even-symmetric and the imaginary part of the FT is odd-symmetric.

In [ ]:
ft_random_signal = fftshift(fft(random_signal))

f, axes = plt.subplots(2, sharex=True, sharey=True)
axes[0].plot(ft_random_signal.real, "o-", color="C0")
axes[1].plot(ft_random_signal.imag, "o-", color="C1")
f.tight_layout()

Taken together, these symmetry properties mean that there is quite a lot of redundancy in the FT of a real signal. A simple way to count the amount of redundancy is to compare the number of samples. Take a real-valued input signal with 64 samples. Computing its FFT gives a total of 128 samples (half in the real part and half in the imaginary part), a factor of 2 redundant.

## Parseval's Theorem

"The sum of squared values over space domain equals sum of squared values over frequency domain."

In [ ]:
assert np.allclose(np.sum(gaussian ** 2), np.sum(ft_gaussian ** 2) / n)

## Circular Shifting

If we shift the signal as if it were periodic (i.e., translate the signal, wrapping around at the edges), this does not affect the Fourier transform magnitude:

In [ ]:
ft_gauss = fftshift(fft(ifftshift(gaussian)))
ft_shift_gauss = fftshift(fft(ifftshift(np.roll(gaussian, 3))))

mag_ft_gauss = np.abs(ft_gauss)
mag_ft_shift_gauss = np.abs(ft_shift_gauss)

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(n_range - n / 2, mag_ft_gauss, "o-")
axes[1].plot(n_range - n / 2, mag_ft_shift_gauss, "o-")

assert np.allclose(mag_ft_gauss, mag_ft_shift_gauss)

## Differentiation

Taking a derivative of a signal in time is the same as multiplying by an imaginary ramp in frequency. In particular, $$\mathrm{Fourier}\Big(\frac{d}{dx} f(x)\Big) = -i\,w\,\mathrm{Fourier}\Big(f(x)\Big),$$ where $i = \sqrt{-1}$ and $w$ is normalized frequency.

For an intuition for the derivative property, recall that $$\frac{d}{dx}\cos{wx} = -w\sin{wx}.$$

For example, consider a Gaussian and the first derivative of a Gaussian.

In [ ]:
gaussian = np.exp(-((n_range - n / 2) ** 2) / (2 * 4 ** 2))
gaussian_deriv = -2 / (2 * 4 ** 2) * (n_range - n / 2) * gaussian

ft_gaussian = fftshift(fft(ifftshift(gaussian)))
ft_gaussian_deriv = fftshift(fft(ifftshift(gaussian_deriv)))

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(n_range - n / 2, gaussian, "o-")
axes[0].plot(n_range - n / 2, gaussian_deriv, "o-")

axes[1].plot(n_range - n / 2, ft_gaussian.real, "o-")
axes[1].plot(n_range - n / 2, ft_gaussian_deriv.imag, "o-")
f.tight_layout()

In [ ]:
ramp = 2 * np.pi / n * 1j * (n_range - n / 2)  # 1j is i, or sqrt(-1)
ft_gaussian_mul_ramp = ramp * ft_gaussian
assert np.allclose(ft_gaussian_deriv, ft_gaussian_mul_ramp, atol=1e-3)

axes[1].plot(n_range - n / 2, ft_gaussian_mul_ramp.imag, "o", color="C2")
display(f)

## Modulation Theorem

Multiplication in time domain is the same as convolution in the frequency domain, up to a known scale factor.  For example, a Gabor function is a sinusoid multiplied by a Gaussian window. Thus, the FT of a Gabor is the convolution of the FT of a Gaussian with the FT of a sinusoid. This is an easy way tto gain an intuition for the filtering properties of a gabor filter.

In [ ]:
gabor = gaussian * sinusoid
ft_gabor = fftshift(fft(ifftshift(gabor)))

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(n_range - n / 2, gabor, "o-")
axes[1].plot(n_range - n / 2, ft_gabor.imag, "o-")
f.tight_layout()

In [ ]:
ft_sinusoid = fftshift(fft(ifftshift(sinusoid)))
axes[1].plot(n_range - n / 2, ft_sinusoid.imag, "o-")

ft_gaussian = fftshift(fft(ifftshift(gaussian)))
axes[1].plot(n_range - n / 2, ft_gaussian.imag, "o-")

display(f)

In [ ]:
conv_of_fts = ndimage.convolve(ft_gaussian.real, ft_sinusoid.imag, mode="wrap") / n
axes[1].plot(n_range - n / 2, conv_of_fts, "*")
assert np.allclose(ft_gabor.imag, conv_of_fts)
display(f)

## Convolution Theorem

Convolution in the time domain is the same as multiplication in the frequency domain, up to a known scale factor. This theorem is extremely useful. Sometimes, you have a filter that is simple to characterize in the frequency domain but complicated in the time domain. For example, it may be comtact in the frequency domain (i.e., nearly zero everywhere), but very big (i.e., lots of samples needed) in the time domain. In such cases, you can do the filtering by Fourier transforming the signal, multiplying in the frequency domain, and then Fourier transforming back.

In [ ]:
from numpy.fft import fftfreq

In [ ]:
gabor_filter = gabor.copy()
impulse_x = np.arange(128)
impulse_x_ctr = impulse_x - impulse_x.size / 2
impulse_signal = (impulse_x == 64).astype(float)
impulse_response = ndimage.convolve(impulse_signal, gabor_filter, mode="wrap")

f, ax = plt.subplots()
ax.plot(impulse_x, impulse_response)
f.tight_layout()

In [ ]:
random_signal = .5 - np.random.rand(impulse_signal.size)
filtered_signal = ndimage.convolve(random_signal, gabor_filter, mode="wrap")
f, ax = plt.subplots()
ax.plot(impulse_x, filtered_signal)
f.tight_layout()

In [ ]:
ft_filtered_signal = fftshift(fft(ifftshift(filtered_signal)))
f, ax = plt.subplots()
ax.plot(impulse_x_ctr, np.abs(ft_filtered_signal))
f.tight_layout()

In [ ]:
frequency_response = fftshift(fft(ifftshift(impulse_response)))
f, ax = plt.subplots()
ax.plot(impulse_x_ctr, np.abs(frequency_response))
f.tight_layout()

In [ ]:
ft_random_signal = fftshift(fft(ifftshift(random_signal)))
product_of_fts = frequency_response * ft_random_signal
assert np.allclose(ft_filtered_signal, product_of_fts)
f, ax = plt.subplots()
ax.plot(impulse_x_ctr, np.abs(product_of_fts))
f.tight_layout()

## Frequency Response

Since the Convolution Theorem is so useful, the Fourier transform of the impulse response of a time-invariant linear system has a special name. It is called the Frequency Response of the linear system.

Remember that for a sinusoidal input, the output of a time-invariant linear system is sinusoidal with the same frequency. Only the amplitude and phase will be changed by filtering. The frequency response of a filter can be used to "read-off" of the amplitude attenuation and the phase shift, for each frequency. For a complicated signal, that can be expressed as the sum of a number of sinusoids, the frequency response can be used to "read-off" the attenuation and phase shift for each component.

As anotherr example, let's compute the frequency response of the one sample delay system. For the delay system, the magnitude of the frequency response is constant (1 for all frequencies), and the phase is $-w$).

In [ ]:
delay_filter = [0, 0, 1]
impulse_response = ndimage.convolve(impulse_signal, delay_filter, mode="wrap")
frequency_response = fftshift(fft(ifftshift(impulse_response)))
mag_frequency_response = np.abs(frequency_response)
phase_frequency_response = np.angle(frequency_response)

f, axes = plt.subplots(2, sharex=True)
axes[0].plot(impulse_x_ctr, mag_frequency_response)
axes[1].plot(impulse_x_ctr, phase_frequency_response)
f.tight_layout()

You may have noticed in a few places, we multiplied by the square root of the number of samples (e.g. 32). This scale factor is needed given the way that the fft is implemented in `numpy`. In some textbooks (e.g., Oppenheim and Schafer), the discrete Fourier transform (DFT) is dfined so that you divide by the number of samples (N) when doing the inverse transform (from the frequency domain back into the space/time domain). In other texts, the DFT is dfined so that youdivded by N when doing the forward transform. In still other texts, you divided by $\sqrt{N}$ when doing both the forward and the inverse transforms, so that the Fourier transform is an orthonormal transform. The implementation in `numpy` follows the first (divide by N in the inverse transform) of these conventions.

If you lose track of which convention is being used, it is, unfortunately, easy to get confused. For example, using the `numpy` convention, we write Parseval's theorem as follows (with a factor of $\frac{1}{N}$):

$$\sum{x[n]^2} = \frac{1}{N}\sum{X[w]^2}.$$

Using the orthonormal (divide by root-N for the forward and inverse transforms) convention, Parseval's theorem has no scale factor:

$$\sum{x[n]^2} = \sum{X[w]^2}.$$

Using the `numpy` convention, the convolution theorem has no scale factor:

$$F(x_1[n] * x_2[n]) = X_1[k]\,X_2[k]$$.

Using the orthonormal convention, the convolution theorem has a factor of $\sqrt(N)$:

$$F(x_1[n] * x_2[n]) = \sqrt(N)\,X_1[k]\,X2[k].$$

## Discrete Cosine Transform

the DCT is another linear transform that is closely related to the DFT. The rows of the DCT transform matrix are cosines:

$$c(k) \,\frac{1}{\sqrt{n}}\,\cos{\frac{\pi}{2N}\,k\,(2n+1)}$$

Where $c(k)=1$ for $k=0$ and $c(k)=\sqrt{2}$ otherwise. Here, $k$ indexes the row and $n$ indexes the column. In other words, $n$ indexes the sample position of the original sample, and $k$ indexes the transform coefficients.

In [ ]:
N = 16
dct_mat = np.zeros((N, N))
nnrange = np.arange(N)
kkrange = nnrange[:, np.newaxis]
cs = np.where(kkrange > 0, np.sqrt(2), 1)
dct_mat = cs / np.sqrt(N) * np.cos(np.pi / (2 * N) * kkrange * (2 * nnrange + 1))

f, ax = plt.subplots(figsize=(4, 4))
ax.matshow(dct_mat, cmap="gray")
f.tight_layout()

The DCT matrix is square and orthonormal:

In [ ]:
assert np.allclose(dct_mat @ dct_mat.T, np.eye(N))
assert np.allclose(dct_mat.T @ dct_mat, np.eye(N))